#Install packages and load libraries

Import common packages

In [ ]:
%%capture
import argparse, cProfile, datetime
from functools import reduce
import glob
import itertools as it
from multiprocessing import cpu_count, Pool
import networkx as nx
import numpy as np
import os

import pandas as pd
import pickle
import matplotlib.pyplot as plt
import random, re, shutil
from scipy.stats import hmean
from sklearn.ensemble import RandomForestClassifier
import sklearn.metrics
from sklearn.metrics import auc, average_precision_score, precision_recall_curve
from sklearn.model_selection import KFold, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
import seaborn as sns
import subprocess as sp
import sys
import tempfile as tf
import time
from tqdm import tqdm as progressMonitor

Enable Google Colab, mount drives, and load proprietary modules...

In [ ]:
%%capture
from google.colab import drive, files, output
drive.mount('/content/drive', force_remount=True)
!rm -r sample_data/

In [ ]:
#useful to keep track of sys vars so as to better monitor space remaining
print(cpu_count())
sysVars = list(globals().keys())

8


In [ ]:
rootDir = '/content/drive/My Drive/'
workDir = rootDir + 'elcfs_protein_complex_modeling/'  #phase 2 directory
workDir_elcfs = rootDir + 'Primary Research/proteinPairs_complexMaps/' #modeling library
workDir_ph1 = rootDir + 'Primary Research/JLMwSCBC_notebook/'  #phase 1 directory

sys.path.insert(0, rootDir)
for p in workDir, workDir_elcfs, workDir_ph1: sys.path.append(p)

In [ ]:
from util import ppiPrediction_v2, dataProcessing, modelEvaluating
from utils import operations, reference, alertMe
pushoverKey_user = 'uith8rmy2npjj1oqpjwcanow3un984'
pushoverAPI = 'aw4v3424kaznrw598r6qge9icddwg7'

In [ ]:
def plotPanels(prjName, panels, outputDir):
  filename = '{0}_Pr-R_Performance'.format(prjName)
  plt.clf()
  fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(24.25, 6))

  for results in panels[0]:
    precRecall_pairs = \
      [dataProcessing.loadResults_precRecall(results[0], results[1])]

    for precRecall_pair in precRecall_pairs:
      precRecall_auc = auc(precRecall_pair[0], precRecall_pair[1])
      ax1.plot(precRecall_pair[0], precRecall_pair[1],
              label=precRecall_pair[2] + ', AUC: {0: .3f}'.format(precRecall_auc))
      ax1.legend(bbox_to_anchor=(0, 0), loc='lower left', ncol=1)
      ax1.set_xlabel('Recall')
      ax1.set_ylabel('Precision')

  for results in panels[1]:
    precRecall_pairs = \
      [dataProcessing.loadResults_precRecall(results[0], results[1])]

    for precRecall_pair in precRecall_pairs:
      precRecall_auc = auc(precRecall_pair[0], precRecall_pair[1])
      ax2.plot(precRecall_pair[0], precRecall_pair[1],
              label=precRecall_pair[2] + ', AUC: {0: .3f}'.format(precRecall_auc))
      ax2.legend(bbox_to_anchor=(0, 0), loc='lower left', ncol=1)
      ax2.set_xlabel('Recall')
      ax2.set_ylabel('Precision')

  for results in panels[2]:
    precRecall_pairs = \
      [dataProcessing.loadResults_precRecall(results[0], results[1])]

    for precRecall_pair in precRecall_pairs:
      precRecall_auc = auc(precRecall_pair[0], precRecall_pair[1])
      ax3.plot(precRecall_pair[0], precRecall_pair[1],
              label=precRecall_pair[2] + ', AUC: {0: .3f}'.format(precRecall_auc))
      ax3.legend(bbox_to_anchor=(0, 0), loc='lower left', ncol=1)
      ax3.set_xlabel('Recall')
      ax3.set_ylabel('Precision')

  filename_wPath = \
    outputDir + filename + '.svg'
  plt.savefig(filename_wPath, bbox_inches='tight', dpi=600)

#Generate predictions

In [ ]:
cvFold = 0

In [ ]:
dataMat_filename = workDir + 'featureData/pairLevel/featMat_agg'

In [ ]:
%%capture
#get models and evaluate
precWeighted_drewFeats, recWeighted_drewFeats = \
  ppiPrediction_v2.eval(workDir, dataMat_filename, cvFold=cvFold)

In [ ]:
#put this block after any cell expected to take a long time
cmdReport = alertMe.statusCheck(pushoverAPI, pushoverKey_user)
cmdReport.finishPush()